P = int(input("Enter starting principle please. "))
n = int(input("Enter number of compounding periods per year. "))
r = float(input("Enter annual interest rate. e.g. 15 for 15% "))
y = int(input("Enter the amount of years. "))

FV = P * (((1 + ((r/100.0)/n)) ** (n*y)))

print ("The final amount after", y, "years is", FV)

In [8]:
import numpy as np
import pandas as pd
from bokeh.io import curdoc, output_notebook
#output_notebook()

from bokeh.models import ColumnDataSource, HoverTool, Span, Legend, Patch
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import brewer

import panel as pn

pn.extension()

def calc(starting_principle: float, 
         contribution_per_compounding_period: float,
         compounding_periods_per_year: int = 1, 
         interest_rate: float = 0.05,
         num_years: int = 10) -> np.ndarray:
    
    # cleaning up user input
    starting_principle = max(0, starting_principle)
    contribution_per_compounding_period = max(0, contribution_per_compounding_period)
    compounding_periods_per_year = max(1, compounding_periods_per_year)
    interest_rate = max(0, interest_rate)
    num_years = max(1, num_years)
    
    n_points = int(num_years * compounding_periods_per_year)
    contributions = np.array(
        [starting_principle + i * contribution_per_compounding_period for i in range(n_points)],
        dtype = float
    )
    accrued_gains = np.zeros_like(contributions)
    gains = np.zeros_like(contributions)
    total = np.zeros_like(contributions)
    
    for i in range(n_points):
        if i == 0:
            total[i] = starting_principle
            gains[i] = 0.0
            accrued_gains[i] = 0.0
        else:
            gain = interest_rate * total[i-1]
            gains[i] = gain
            accrued_gains[i] = accrued_gains[i-1] + gain
            total[i] = total[i-1] + contribution_per_compounding_period + gain
    
    df = pd.DataFrame({
        "years_elapsed": np.arange(0, num_years, 1/compounding_periods_per_year),
        "contributions": contributions,
        "accrued_gains": accrued_gains,
        "gains": gains,
        "total": total,
    })
    
    return df
  
def find_indices_of_multiples(starting_capital: float, df: pd.DataFrame):
    as_factor = df.total.values // starting_capital
    return 1+np.where(as_factor[1:] != as_factor[:-1])[0]

def find_doubling_points(df: pd.DataFrame,
                         starting_capital: float,
                         contribution_per_compounding_period: float) -> pd.DataFrame:
    starting_capital = contribution_per_compounding_period if starting_capital == 0 else starting_capital
    as_factor = df.total.values % starting_capital
    doubled = find_indices_of_multiples(
        contribution_per_compounding_period if starting_capital == 0 else starting_capital,
        df
    )
    return df.iloc[doubled]

def find_overtaking_point(df: pd.DataFrame,
                          contribution_per_compounding_period: float) -> pd.DataFrame:
    overtaking_idx = np.where(df.gains > contribution_per_compounding_period)[0]
    if overtaking_idx.shape[0] > 0:
        return df.iloc[overtaking_idx]
    else:
        return pd.DataFrame()

def find_points_of_interest(df: pd.DataFrame,
                            starting_capital: float,
                            contribution_per_compounding_period: float):
    doubling_points_df = find_doubling_points(
        df,
        starting_capital,
        contribution_per_compounding_period
    )
    overtaking_point_df = find_overtaking_point(
        df,
        contribution_per_compounding_period
    )
    return doubling_points_df, overtaking_point_df
    

In [63]:
def setup_parameters(which: str = 'left'):
    starting_principle = pn.widgets.FloatInput(
        name=f'Starting Capital', 
        value=100.0, 
        step=1e-1, 
        start=0, 
        end=1e9
    )

    contribution_per_compounding_period = pn.widgets.FloatInput(
        name=f'Contribution per Compounding Period', 
        value=100.0, 
        step=1e-1, 
        start=0, 
        end=1e9
    )

    compounding_periods_per_year = pn.widgets.IntInput(
        name=f'Compounding Periods per Year', 
        value=1, 
        step=1, 
        start=1, 
        end=52
    )

    interest_rate = pn.widgets.FloatInput(
        name=f'Interest Rate', 
        value=0.04, 
        step=1e-3, 
        start=0, 
        end=1.0
    )

    num_years = pn.widgets.IntInput(
        name=f'Number of Years', 
        value=10, 
        step=1, 
        start=1, 
        end=99
    )

    return  starting_principle, \
            contribution_per_compounding_period, \
            compounding_periods_per_year, \
            interest_rate, \
            num_years


def plot_curve(    starting_capital: float,
                   contribution_per_compounding_period: float,
                   compounding_periods_per_year: int = 1, 
                   interest_rate: float = 0.05,
                   num_years: int = 10):

    df = calc(
        starting_capital,
        contribution_per_compounding_period,
        compounding_periods_per_year = compounding_periods_per_year,
        interest_rate = interest_rate,
        num_years = num_years
    )

    #curdoc().theme = 'dark_minimal'

    p = figure(
        x_range=(0, df.years_elapsed.max()), 
        y_range=(0, 1.1*df.total.max()),
        height=300
    )

    v_areas = p.varea_stack(
        stackers=['contributions', 'accrued_gains'], 
        x='years_elapsed', 
        color=brewer['Spectral'][10][3:5], 
        legend_label=['Contributions', 'Accrued Gains'], 
        source=df,
        hover_alpha=0.75,
        fill_alpha=0.5,
        hover_color=brewer['Spectral'][10][3:5],
    )
    p.add_tools(
        HoverTool(
            tooltips=[],
            mode='mouse'
        )
    )

    total_line = p.line(
        x='years_elapsed', 
        y='total', 
        line_color='red', 
        source=df,
        legend_label='Total Value',
        line_width=3
    )
    p.add_layout(total_line)
    
    doubling_points_df, overtaking_points_df = find_points_of_interest(
        df,
        starting_capital,
        contribution_per_compounding_period
    )
    if overtaking_points_df.shape[0] > 0:
        
        first = int(overtaking_points_df.years_elapsed.values.min())
        xs = list(
            range(first, round(df.years_elapsed.values.max())+1)
        )
        varea = p.varea(
            x=xs,
            y1=0, 
            y2=[1.1*overtaking_points_df.total.values.max()]*len(xs), 
            alpha=0.1,
            fill_color='blue',
            legend_label='Gains > Contribution'
        )
        p.add_layout(varea)
        
    if doubling_points_df.shape[0] > 0:        
        factor_points = p.scatter(
            'years_elapsed',
            'total',
            source=doubling_points_df,
            fill_alpha=1,
            fill_color='red',
            line_color='white',
            size=12,
            marker='star',
            legend_label='Multiple of Starting Capital'
        )
        p.add_layout(factor_points)
    
      
    
    p.add_tools(HoverTool(
        tooltips=[
            ('Years Since Start',   '@years_elapsed'),
            ('Total Contributions',  '$@contributions'), 
            ('Accrued Gains', '$@accrued_gains'),
            ('Total', '$@total')
        ],
        mode='mouse'
    ))
    
    p.legend.location = "top_left"
    p.legend.orientation = "vertical"
    p.legend.background_fill_color = "#fafafa"

    p.grid.minor_grid_line_alpha = 0
    p.xaxis.axis_label = "Years Since Start"
    p.yaxis.axis_label = "Value"
    p.toolbar.logo = None
    p.toolbar_location = None
    return p
    
spleft, \
cpcpleft, \
cppyleft, \
irleft, \
nyleft = setup_parameters(which='left')

spright, \
cpcpright, \
cppyright, \
irright, \
nyright = setup_parameters(which='right')

@pn.depends(spleft, cpcpleft, cppyleft, irleft, nyleft)
def plot_left( starting_capital: float,
               contribution_per_compounding_period: float,
               compounding_periods_per_year: int = 1, 
               interest_rate: float = 0.05,
               num_years: int = 10):
    return plot_curve(    
        starting_capital,
        contribution_per_compounding_period,
        compounding_periods_per_year = compounding_periods_per_year,
        interest_rate = interest_rate,
        num_years = num_years,
    )

@pn.depends(spright, cpcpright, cppyright, irright, nyright)
def plot_right(starting_capital: float,
               contribution_per_compounding_period: float,
               compounding_periods_per_year: int = 1, 
               interest_rate: float = 0.05,
               num_years: int = 10):
    return plot_curve(    
        starting_capital,
        contribution_per_compounding_period,
        compounding_periods_per_year = compounding_periods_per_year,
        interest_rate = interest_rate,
        num_years = num_years,
    )

In [64]:
app = pn.Row(
    pn.Spacer(background='#FF0000'),
    pn.Column(
        pn.Column('<h2><center>Scenario A</center></h2>', spleft, cpcpleft, cppyleft, irleft, nyleft, align='center'), 
        plot_left
    ),
    pn.Spacer(background='#FF0000'),
    pn.Column(
        pn.Column('<h2><center>Scenario B</center></h2>', spright, cpcpright, cppyright, irright, nyright, align='center'), 
        plot_right
    ),
    pn.Spacer(background='#FF0000'),
)

pn.widgets.Select.sizing_mode = 'stretch_width'
asapp = pn.panel(app, sizing_mode="stretch_width")
asapp.servable()
asapp

[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]


[1 2 3 4 5 6 7 8 9]

Row
    [0] Spacer(background='#FF0000')
    [1] Column
        [0] Column(align='center')
            [0] Markdown(str)
            [1] FloatInput(end=1000000000.0, name='Starting Capital', start=0, value=100.0)
            [2] FloatInput(end=1000000000.0, name='Contribution p..., start=0, value=100.0)
            [3] IntInput(end=52, name='Compounding P..., start=1, value=1)
            [4] FloatInput(end=1.0, name='Interest Rate', start=0, step=0.001, value=0.04)
            [5] IntInput(end=99, name='Number of Years', start=1, value=10)
        [1] ParamFunction(function)
    [2] Spacer(background='#FF0000')
    [3] Column
        [0] Column(align='center')
            [0] Markdown(str)
            [1] FloatInput(end=1000000000.0, name='Starting Capital', start=0, value=100.0)
            [2] FloatInput(end=1000000000.0, name='Contribution p..., start=0, value=100.0)
            [3] IntInput(end=52, name='Compounding P..., start=1, value=1)
            [4] FloatInput(end=1.0, name='Interest Rate', start=0, step=0.001, value=0.04)
            [5] IntInput(end=99, name='Number of Years', start=1, value=10)
        [1] ParamFunction(function)
    [4] Spacer(background='#FF0000')